In [1]:
import torch
import torchtext
from torchtext.datasets import text_classification
NGRAMS = 2
import os
if not os.path.isdir('./.data'):
    os.mkdir('./.data')
train_dataset, test_dataset = text_classification.DATASETS['AG_NEWS'](
    root='./.data', ngrams=NGRAMS, vocab=None)
BATCH_SIZE = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


120000lines [00:07, 15585.52lines/s]
120000lines [00:16, 7383.40lines/s]
7600lines [00:01, 7544.42lines/s]


In [8]:
train_dataset[999]

(3,
 tensor([  1871,     61,   9774,      7,    101,    230,     28,   3770, 811725,
            769,    814,     14,     36,     15,     67, 233818,     85,      2,
              4,      3,  10555,   1888,    214,      4,     24,      6, 116083,
              8,   1795,     16,      6,     72,     19,  20590,  32230,    101,
            115,      2,     14,   4406,     15,    230,      9,  33820,     11,
             30,  26105,      5,  48191,      4,   2306,      9,   2802,      3,
           3372,     30,  82937,      5,    683,      2,      2,      2, 240926,
         479966,  14342,   6250,  74877,  65422, 178979, 809366, 811726,    822,
           4291,     62,     63,    584, 502056, 338958,     89,    253,     42,
         459246, 383932, 404043,   4307,    734,    748, 122122, 271943,  34780,
         848997,    296,   4116,   4993, 457614, 353461, 355168,    981,    126,
            384,  13942,  12119,  20455,   8913, 317615,  41380,  14744, 386125,
          80044,  84344,

In [2]:
import torch.nn as nn
import torch.nn.functional as F
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [3]:
VOCAB_SIZE = len(train_dataset.get_vocab())
EMBED_DIM = 32
NUN_CLASS = len(train_dataset.get_labels())
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUN_CLASS).to(device)

In [4]:
def generate_batch(batch):
    label = torch.tensor([entry[0] for entry in batch])
    text = [entry[1] for entry in batch]
    offsets = [0] + [len(entry) for entry in text]
    # torch.Tensor.cumsum returns the cumulative sum
    # of elements in the dimension dim.
    # torch.Tensor([1.0, 2.0, 3.0]).cumsum(dim=0)

    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text = torch.cat(text)
    return text, offsets, label

In [16]:
d = generate_batch([train_dataset[1], train_dataset[80]])
d[1]

tensor([ 0, 83])

In [28]:
data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True,
                      collate_fn=generate_batch)
x = data.__iter__()
x

In [5]:
from torch.utils.data import DataLoader

def train_func(sub_train_):

    # Train the model
    train_loss = 0
    train_acc = 0
    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True,
                      collate_fn=generate_batch)
    for i, (text, offsets, cls) in enumerate(data):
        optimizer.zero_grad()
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        output = model(text, offsets)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()

    # Adjust the learning rate
    scheduler.step()

    return train_loss / len(sub_train_), train_acc / len(sub_train_)

def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, offsets, cls in data:
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return loss / len(data_), acc / len(data_)

In [6]:
import time
from torch.utils.data.dataset import random_split
N_EPOCHS = 5
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

train_len = int(len(train_dataset) * 0.95)
sub_train_, sub_valid_ = \
    random_split(train_dataset, [train_len, len(train_dataset) - train_len])

for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_func(sub_train_)
    valid_loss, valid_acc = test(sub_valid_)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

Epoch: 1  | time in 0 minutes, 11 seconds
	Loss: 0.0261(train)	|	Acc: 84.7%(train)
	Loss: 0.0002(valid)	|	Acc: 88.3%(valid)
Epoch: 2  | time in 0 minutes, 11 seconds
	Loss: 0.0119(train)	|	Acc: 93.7%(train)
	Loss: 0.0001(valid)	|	Acc: 90.9%(valid)
Epoch: 3  | time in 0 minutes, 11 seconds
	Loss: 0.0069(train)	|	Acc: 96.4%(train)
	Loss: 0.0002(valid)	|	Acc: 90.7%(valid)
Epoch: 4  | time in 0 minutes, 11 seconds
	Loss: 0.0039(train)	|	Acc: 98.0%(train)
	Loss: 0.0002(valid)	|	Acc: 91.0%(valid)
Epoch: 5  | time in 0 minutes, 11 seconds
	Loss: 0.0022(train)	|	Acc: 99.0%(train)
	Loss: 0.0002(valid)	|	Acc: 90.9%(valid)
